In [ ]:
pip install delta-spark==2.3.0

In [ ]:
import pyspark
import delta
from delta.tables import DeltaTable

print("PySpark:", pyspark.__version__)   # 3.3.3
print("Delta:", delta.__version__)       # 2.3.0
print("Spark:", spark.version)           # 3.3.3


In [16]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import *

spark = SparkSession.builder \
    .appName("PJ-Notebook-PoC") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://192.168.2.70:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()
spark.conf.set("parquet.block.size", 134217728)  # 128MB
spark.conf.set("parquet.page.size", 1048576)     # 1MB
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "false")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "false")

spark.sparkContext.setLogLevel("WARN")
print("Spark Session Created Successfully!")

Spark Session Created Successfully!


In [17]:
from pyspark.sql.types import *

KAFKA_BOOTSTRAP = "192.168.2.70:9092"
TOPIC_NAME = "cdc.order.public.AbpOrders"
BRONZE_PATH = "s3a://lakehouse/bronze/abp_orders"
SILVER_PATH = "s3a://lakehouse/silver/orders"
CHECKPOINT_BRONZE = "s3a://lakehouse/checkpoints/bronze/abp_orders"
CHECKPOINT_SILVER = "s3a://lakehouse/checkpoints/silver/orders"

cdc_schema = StructType([
    StructField("before", StringType()),
    StructField("after", StructType([
        StructField("Id", LongType()),
        StructField("OrderNumber", StringType()),
        StructField("StoreId", LongType()),
        StructField("CustomerId", StringType()),
        StructField("OrderTotal", StringType()),
        StructField("CreationTime", LongType()),
        StructField("OrderStatus", IntegerType()),
        StructField("IsDeleted", BooleanType())
    ])),
    StructField("op", StringType()),
    StructField("ts_ms", LongType())
])

print("Configurations & Schema Ready")

Configurations & Schema Ready


In [18]:
print(f"Reading from Kafka Topic: {TOPIC_NAME}...")

raw_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP) \
    .option("subscribe", TOPIC_NAME) \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

bronze_df = raw_stream.select(
    col("key").cast("string").alias("kafka_key"),
    col("value").cast("string").alias("kafka_value"),
    col("timestamp").alias("kafka_ts"),
    current_timestamp().alias("_ingest_ts")
)

query_bronze = bronze_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", CHECKPOINT_BRONZE) \
    .trigger(availableNow=True) \
    .start(BRONZE_PATH)

query_bronze.awaitTermination()
print("Bronze Ingestion Completed!")

Reading from Kafka Topic: cdc.order.public.AbpOrders...
26/01/14 14:50:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Bronze Ingestion Completed!


In [19]:
try:
    df_bronze_check = spark.read.format("delta").load(BRONZE_PATH)
    count = df_bronze_check.count()
    print(f"Bronze Table Record Count: {count}")
    df_bronze_check.show(5, truncate=False)
except Exception as e:
    print(e)

Bronze Table Record Count: 23790


[Stage 60:===========================================>              (3 + 1) / 4]

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
from py4j.java_gateway import java_import

java_import(spark._jvm, "java.net.URI")
java_import(spark._jvm, "org.apache.hadoop.fs.FileSystem")
java_import(spark._jvm, "org.apache.hadoop.fs.Path")

def delete_s3a_path(path_str):
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    uri = spark._jvm.java.net.URI(path_str)
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(uri, hadoop_conf)
    path = spark._jvm.org.apache.hadoop.fs.Path(uri)

    if fs.exists(path):
        print(f"🧹 Deleting checkpoint: {path_str}")
        fs.delete(path, True)
        print("Deleted successfully")
    else:
        print(f"Path does not exist: {path_str}")


In [21]:

from pyspark.sql.functions import from_json, col, row_number, when, lit
from pyspark.sql.window import Window
from pyspark.sql.types import *
from delta.tables import *

envelope_schema = StructType([
    StructField("schema", StructType([]), True),  # ignore
    StructField("payload", StructType([
        StructField("before", StructType([
            StructField("Id", LongType())
        ]), True),

        StructField("after", StructType([
            StructField("Id", LongType()),
            StructField("OrderNumber", StringType()),
            StructField("StoreId", LongType()),
            StructField("CustomerId", StringType()),
            StructField("OrderTotal", StringType()),
            StructField("CreationTime", LongType()),
            StructField("OrderStatus", IntegerType()),
            StructField("IsDeleted", BooleanType())
        ]), True),

        StructField("op", StringType()),
        StructField("ts_ms", LongType())
    ]), True)
])


def process_silver_batch(df, batch_id):
    batch_count = df.count()
    if batch_count == 0:
        return

    parsed_df = df.select(
        from_json(col("kafka_value"), envelope_schema).alias("envelope")
    )

    valid_df = parsed_df.select(
        col("envelope.payload.after.*"),
        col("envelope.payload.op"),
        col("envelope.payload.ts_ms")
    ).filter(col("Id").isNotNull())

    from pyspark.sql.functions import max_by, struct

    deduped_df = valid_df.groupBy("Id").agg(
        max_by(struct("*"), col("ts_ms")).alias("row")
    ).select("row.*")

    final_df = deduped_df.withColumn(
        "is_deleted_final",
        when(col("op") == 'd', lit(True))
        .when(col("IsDeleted") == True, lit(True))
        .otherwise(lit(False))
    ).coalesce(1)

    if not DeltaTable.isDeltaTable(spark, SILVER_PATH):
        final_df.write.format("delta") \
            .mode("overwrite") \
            .option("path", SILVER_PATH) \
            .saveAsTable("default.orders")
    else:
        DeltaTable.forPath(spark, SILVER_PATH) \
            .alias("t") \
            .merge(final_df.alias("s"), "t.Id = s.Id") \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()

print("Starting Silver Processing (Fixed Schema)...")
import shutil
try:
    delete_s3a_path(SILVER_PATH)
    delete_s3a_path(CHECKPOINT_SILVER)
    pass
except:
    print("Could not clear checkpoint automatically")

query_silver = spark.readStream \
    .format("delta") \
    .load(BRONZE_PATH) \
    .writeStream \
    .foreachBatch(process_silver_batch) \
    .option("checkpointLocation", CHECKPOINT_SILVER) \
    .trigger(availableNow=True) \
    .start()

query_silver.awaitTermination()
print("Silver Processing Completed!")

Starting Silver Processing (Fixed Schema)...
Path does not exist: s3a://lakehouse/silver/orders
Path does not exist: s3a://lakehouse/checkpoints/silver/orders
26/01/14 14:50:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Silver Processing Completed!


In [22]:
# Kiểm tra kết quả cuối cùng
try:
    df_silver = spark.read.format("delta").load(SILVER_PATH)
    print("=== SILVER DATA PREVIEW ===")
    df_silver.select(
        "Id", "OrderNumber", "OrderTotal", "OrderStatus", "is_deleted_final"
    ).show(10)
    
    print(f"Total Silver Records: {df_silver.count()}")
    
    # Kiểm tra thử 1 record cụ thể nếu biết ID
    # df_silver.filter("Id = 12345").show()
    
except Exception as e:
    print(f"Lỗi đọc bảng Silver: {e}")

=== SILVER DATA PREVIEW ===
+----+-----------+----------+-----------+----------------+
|  Id|OrderNumber|OrderTotal|OrderStatus|is_deleted_final|
+----+-----------+----------+-----------+----------------+
|  26| 240HCB1WFA|  32000.00|        150|           false|
|  29| NG6QKOLEVV|  32000.00|        150|           false|
| 474| TTEG9S72T8|  70000.00|         20|           false|
|1677|   MBAK0097|  80000.00|         70|           false|
|1697|   MBAK0117|  85000.00|         20|           false|
|1806|   MBAK0196|  80000.00|          5|           false|
|1950|   MBAK0343| 105000.00|         70|           false|
|2040|   MBAK0439| 140000.00|         70|           false|
|2214|   MBAK0617|  84000.00|         70|           false|
|2250|   MBAK0653|  84000.00|         70|           false|
+----+-----------+----------+-----------+----------------+
only showing top 10 rows

Total Silver Records: 7930
